# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [97]:
# Your solution goes here
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [98]:
url="https://www.allocine.fr/films/decennie-1980/annee-1987/"

soup=BeautifulSoup(requests.get(url).content, 'html.parser')

titles=[]
years=[]
runtime=[]
genres=[]


film_cards= soup.find_all("div", class_="card entity-card entity-card-list cf")
film_name=""
for film_card in film_cards:
    if film_card.find("a",class_='meta-title-link') is not None:
        film_name=film_card.find("a",class_='meta-title-link').text.strip(" ")
        titles.append(film_name)

        film_year=film_card.find("span", class_="date").text.strip()
        years.append(film_year)

        film_info=film_card.find("div", class_="meta-body-info")
        info=film_info.text.split("|")
        print(info)
        
        info1=info[1].replace("\n","")
        info2=info[2].replace("\n","")
        runtime.append(info1)
        genres.append(info2)

df =pd.DataFrame({'titles':titles, 'release date':years, 'runtime':runtime, 'genres':genres})
display (df)

['\n16 décembre 1987\n', '\n2h 00min\n', '\nAventure,\nComédie,\nFantastique\n']
['\n20 janvier 1988\n', '\n1h 43min\n', '\nAction,\nPolicier,\nScience Fiction\n']
['\n19 août 1987\n', '\n1h 47min\n', '\nAction,\nScience Fiction\n']
['\n23 septembre 1987\n', '\n2h 08min\n', '\nDrame,\nFantastique,\nRomance\n']
['\n2 mars 1998\n', '\n1h 23min\n', '\nAventure,\nAnimation,\nScience Fiction\n']
['\n6 décembre 1989\n', '\n1h 37min\n', '\nDrame,\nFantastique,\nRomance\n']
['\n9 mars 1988\n', '\n1h 38min\n', '\nAventure,\nFantastique\n']
['\n21 octobre 1987\n', '\n1h 56min\n', '\nDrame,\nGuerre\n']
['\n7 septembre 1988\n', '\n2h 03min\n', '\nComédie,\nDrame,\nFamille\n']
['\n23 décembre 1987\n', '\n1h 40min\n', '\nMusical,\nRomance\n']
['\n21 octobre 1987\n', '\n1h 59min\n', '\nPolicier\n']
['\n2 mars 1988\n', '\n2h 51min\n', '\nDrame,\nRomance\n']
['\n11 mars 1987\n', '\n1h 35min\n', '\nComédie,\nErotique\n']
['\n10 février 1988\n', '\n2h 02min\n', '\nDrame\n']
['\n16 mars 1988\n', '\n2h 34m

,titles,release date,runtime,genres
0,L'Aventure intérieure,16 décembre 1987,2h 00min,"Aventure,Comédie,Fantastique"
1,Robocop,20 janvier 1988,1h 43min,"Action,Policier,Science Fiction"
2,Predator,19 août 1987,1h 47min,"Action,Science Fiction"
3,Les Ailes du désir,23 septembre 1987,2h 08min,"Drame,Fantastique,Romance"
4,Gandahar,2 mars 1998,1h 23min,"Aventure,Animation,Science Fiction"
5,Rouge,6 décembre 1989,1h 37min,"Drame,Fantastique,Romance"
6,Princess Bride,9 mars 1988,1h 38min,"Aventure,Fantastique"
7,Full Metal Jacket,21 octobre 1987,1h 56min,"Drame,Guerre"
8,Big,7 septembre 1988,2h 03min,"Comédie,Drame,Famille"
9,Dirty Dancing,23 décembre 1987,1h 40min,"Musical,Romance"


In [99]:
url="https://www.allocine.fr/films/decennie-1980/annee-1987/"

def allocine_df(url):
    soup=BeautifulSoup(requests.get(url).content, 'html.parser')

    titles=[]
    years=[]
    runtime=[]
    genres=[]


    film_cards= soup.find_all("div", class_="card entity-card entity-card-list cf")
    film_name=""
    for film_card in film_cards:
     if film_card.find("a",class_='meta-title-link') is not None:
            film_name=film_card.find("a",class_='meta-title-link').text.strip(" ")
            titles.append(film_name)

            film_year=film_card.find("span", class_="date").text.strip()
            years.append(film_year)

            film_info=film_card.find("div", class_="meta-body-info")
            info=film_info.text.split("|")
        
            info1=info[1].replace("\n","")
            info2=info[2].replace("\n","")
            runtime.append(info1)
            genres.append(info2)

    df =pd.DataFrame({'titles':titles, 'release date':years, 'runtime':runtime, 'genres':genres})
    return df
display (df)



,titles,release date,runtime,genres
0,L'Aventure intérieure,16 décembre 1987,2h 00min,"Aventure,Comédie,Fantastique"
1,Robocop,20 janvier 1988,1h 43min,"Action,Policier,Science Fiction"
2,Predator,19 août 1987,1h 47min,"Action,Science Fiction"
3,Les Ailes du désir,23 septembre 1987,2h 08min,"Drame,Fantastique,Romance"
4,Gandahar,2 mars 1998,1h 23min,"Aventure,Animation,Science Fiction"
5,Rouge,6 décembre 1989,1h 37min,"Drame,Fantastique,Romance"
6,Princess Bride,9 mars 1988,1h 38min,"Aventure,Fantastique"
7,Full Metal Jacket,21 octobre 1987,1h 56min,"Drame,Guerre"
8,Big,7 septembre 1988,2h 03min,"Comédie,Drame,Famille"
9,Dirty Dancing,23 décembre 1987,1h 40min,"Musical,Romance"


In [100]:
url2="https://www.allocine.fr/films/decennie-1980/annee-1987/?page=3"

df3=allocine_df(url2)
display(df3)

,titles,release date,runtime,genres
0,L'Eté en pente douce,29 avril 1987,1h 36min,Drame
1,Bad Taste,24 août 1988,1h 32min,"Action,Comédie,Epouvante-horreur"
2,Sens unique,16 décembre 1987,1h 50min,"Action,Drame,Romance"
3,Génération perdue,28 janvier 1998,1h 38min,"Comédie,Epouvante-horreur"
4,Sous le soleil de Satan,2 septembre 1987,1h 43min,Drame
5,Tandem,17 juin 1987,1h 31min,Comédie dramatique
6,La Folle Histoire de l'espace,14 octobre 1987,1h 36min,"Aventure,Comédie,Science Fiction"
7,Hidden,24 juin 2016,1h 40min,"Action,Epouvante-horreur"
8,Prince des ténèbres,20 avril 1988,1h 42min,Epouvante-horreur
9,Street Trash,24 juin 1987,1h 40min,Epouvante-horreur


In [105]:
df_list=list()
for i in range (1,5):
    url=("https://www.allocine.fr/films/decennie-1980/annee-1987/?page="+str(i))
    df__=allocine_df(url)
    df_list.append(df__)

df_all=pd.concat(df_list).reset_index(drop=True)
display(df_all)


,titles,release date,runtime,genres
0,L'Aventure intérieure,16 décembre 1987,2h 00min,"Aventure,Comédie,Fantastique"
1,Robocop,20 janvier 1988,1h 43min,"Action,Policier,Science Fiction"
2,Predator,19 août 1987,1h 47min,"Action,Science Fiction"
3,Les Ailes du désir,23 septembre 1987,2h 08min,"Drame,Fantastique,Romance"
4,Gandahar,2 mars 1998,1h 23min,"Aventure,Animation,Science Fiction"
5,Rouge,6 décembre 1989,1h 37min,"Drame,Fantastique,Romance"
6,Princess Bride,9 mars 1988,1h 38min,"Aventure,Fantastique"
7,Full Metal Jacket,21 octobre 1987,1h 56min,"Drame,Guerre"
8,Big,7 septembre 1988,2h 03min,"Comédie,Drame,Famille"
9,Dirty Dancing,23 décembre 1987,1h 40min,"Musical,Romance"


In [102]:
url="https://www.allocine.fr/films/decennie-1980/annee-1987/"
soup=BeautifulSoup(requests.get(url).content, 'html.parser')

titles=[]
years=[]
ratings_press=[]
runtime=[]
genres=[]
descriptions=[]
directors=[]
stars=[]
votes_public=[]
gross=[]

film_cards= soup.find_all("div", class_="card entity-card entity-card-list cf")
print(film_cards)

[<div class="card entity-card entity-card-list cf">
<figure class="thumbnail">
<span class="ACrL3ZACrpZGVvL3BsYXllcl9nZW5fY21lZGlhPTE5NDI3OTc3JmNmaWxtPTI5NjguaHRtbA== thumbnail-container thumbnail-link" title="Bande-annonce L'Aventure intérieure">
<img alt="Bande-annonce L'Aventure intérieure" class="thumbnail-img" height="420" src="https://fr.web.img2.acsta.net/c_310_420/medias/nmedia/18/36/15/89/19631793.jpg" width="310"/>
<span class="ico-play-wrapper">
<span class="icon icon-play-arrow"></span>
</span>
</span>
</figure>
<div class="meta meta-affintiy-score">
<h2 class="meta-title">
<a class="meta-title-link" href="/film/fichefilm_gen_cfilm=2968.html">L'Aventure intérieure</a>
</h2>
<div class="meta-body">
<div class="meta-body-item meta-body-info">
<span class="date">16 décembre 1987</span>
<span class="spacer">|</span>
2h 00min
<span class="spacer">|</span>
<span clas="dark-grey-link" class="ACrL2ZACrpbG1zL2dlbnJlLTEzMDAxLw==">Aventure</span>,
<span clas="dark-grey-link" class="AC

## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

In [103]:
# Your solution goes here